In [1]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 48.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.27.59
    Uninstalling botocore-1.27.59:
      Successfully uninstalled botocore-1.27.59
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.109 which is incompatible.


In [2]:
!cp -r /kaggle/input/datasetuno/transistor/* /kaggle/working

In [3]:
import boto3
import uuid, os

ACCESS_KEY = os.environ.get('AWS_ACCESS_KEY_ID')
SECRET_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')
BUCKET_NAME = os.environ.get('BUCKET_NAME')

DATASET_NAME = "transistor"
GENERATOR_NAME = "generator_weights.pt"
GENERATOR_PATH = f"{DATASET_NAME}/{GENERATOR_NAME}"
DESCRIMINATOR_NAME = "descriminator_weights.pt"
DESCRIMINATOR_PATH = f"{DATASET_NAME}/{DESCRIMINATOR_NAME}"
IMAGES_NAME = "images.zip"
session = boto3.Session(
    aws_access_key_id=ACCESS_KEY, 
    aws_secret_access_key=SECRET_KEY
)

s3 = session.resource('s3')
bucket = s3.Bucket(BUCKET_NAME)

## DOWNLOADING THE GENERATOR

In [4]:
try:
    bucket.download_file(Key=GENERATOR_PATH, Filename=GENERATOR_NAME)
except:
    print("The generator weights don't exist.")

The generator weights don't exist.


## DOWNLOADING THE DESCRIMINATOR

In [5]:
try:
    bucket.download_file(Bucket=BUCKET_NAME, Key=DESCRIMINATOR_PATH, Filename=DESCRIMINATOR_NAME)
except:
    print("The descriminator weights don't exist.")

The descriminator weights don't exist.


In [6]:
!rm -rf images/*

## DCGAN Architecture

In [7]:
!python dcgan.py --n_epochs 8000 --batch_size 64 --lr 0.0002 --b1 0.5 --b2 0.999 --n_cpu 8 --latent_dim 100 --img_size 256 --channels 3 --sample_interval 50 --save_weights True --augment_data True --load_weights True --save_path "augment" --count 100 --weights_filename "weights.pt"

Namespace(augment_data=True, b1=0.5, b2=0.999, batch_size=64, channels=3, count=100, img_size=256, latent_dim=100, load_weights=True, lr=0.0002, n_cpu=8, n_epochs=8000, sample_interval=50, save_path='augment', save_weights=True, train_data=True, weights_filename='weights.pt')
No checkpoint found at generator_weights.pt
No checkpoint found at descriminator_weights.pt
[Epoch 0/8000] [Batch 0/1] [D loss: 0.693126] [G loss: 0.695485]
[Epoch 1/8000] [Batch 0/1] [D loss: 0.692680] [G loss: 0.694819]
[Epoch 2/8000] [Batch 0/1] [D loss: 0.692115] [G loss: 0.694310]
[Epoch 3/8000] [Batch 0/1] [D loss: 0.691036] [G loss: 0.694118]
[Epoch 4/8000] [Batch 0/1] [D loss: 0.689397] [G loss: 0.694176]
[Epoch 5/8000] [Batch 0/1] [D loss: 0.686691] [G loss: 0.694738]
[Epoch 6/8000] [Batch 0/1] [D loss: 0.682020] [G loss: 0.695388]
[Epoch 7/8000] [Batch 0/1] [D loss: 0.675112] [G loss: 0.696127]
[Epoch 8/8000] [Batch 0/1] [D loss: 0.664534] [G loss: 0.696490]
[Epoch 9/8000] [Batch 0/1] [D loss: 0.647685] 

## INFOGAN Architecture

In [ ]:
#!python infogan.py --n_epochs 5000 --batch_size 64 --lr 0.0002 --b1 0.5 --b2 0.999 --n_cpu 8 --latent_dim 100 --n_classes 1 --img_size 256 --channels 3 --sample_interval 50 --save_weights True --augment_data True --load_weights True --save_path "augment" --count 100 --weights_filename "weights.pt"

## Upload or Overwrite the Generator weights

In [8]:
try:
    bucket.upload_file(Key=GENERATOR_PATH, Filename=GENERATOR_NAME)
    print(f'File {GENERATOR_NAME} uploaded to S3 bucket {BUCKET_NAME}')
except Exception as e:
    print(f"Something went wrong: {e}")

Something went wrong: Failed to upload generator_weights.pt to transistor/transistor/generator_weights.pt: An error occurred (AccessDenied) when calling the CreateMultipartUpload operation: Access Denied


## Upload or Overwrite the Descriminator weights

In [9]:
try:
    bucket.upload_file(Key=DESCRIMINATOR_PATH, Filename=DESCRIMINATOR_NAME)
    print(f'File {DESCRIMINATOR_NAME} uploaded to S3 bucket {BUCKET_NAME}')
except:
    print("Something went wrong")

Something went wrong


In [10]:
!zip images.zip images/*

  adding: images/0.png (deflated 0%)
  adding: images/100.png (deflated 0%)
  adding: images/1000.png (deflated 0%)
  adding: images/1050.png (deflated 0%)
  adding: images/1100.png (deflated 0%)
  adding: images/1150.png (deflated 0%)
  adding: images/1200.png (deflated 0%)
  adding: images/1250.png (deflated 0%)
  adding: images/1300.png (deflated 0%)
  adding: images/1350.png (deflated 0%)
  adding: images/1400.png (deflated 0%)
  adding: images/1450.png (deflated 0%)
  adding: images/150.png (deflated 0%)
  adding: images/1500.png (deflated 0%)
  adding: images/1550.png (deflated 0%)
  adding: images/1600.png (deflated 0%)
  adding: images/1650.png (deflated 0%)
  adding: images/1700.png (deflated 0%)
  adding: images/1750.png (deflated 0%)
  adding: images/1800.png (deflated 0%)
  adding: images/1850.png (deflated 0%)
  adding: images/1900.png (deflated 0%)
  adding: images/1950.png (deflated 0%)
  adding: images/200.png (deflated 0%)
  adding: images/2000.png (deflated 0%)
  addi

## Upload or Overwrite the Training Images

In [11]:
IMAGES_PATH = f"{DATASET_NAME}/images-{uuid.uuid4()}.zip"
try:
    bucket.upload_file(Key=IMAGES_PATH, Filename=IMAGES_NAME)
    print(f'File {IMAGES_NAME} uploaded to S3 bucket {BUCKET_NAME}')
except:
    print("Something went wrong")

Something went wrong


## CREATE GIF

In [12]:
import os
import imageio
import numpy as np
from PIL import Image, ImageDraw

def create_gif(input_dir, output_file, fps=2):
    # Collect image files from the input directory
    image_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) 
                   if os.path.isfile(os.path.join(input_dir, f)) and f.endswith('.png')]
    # Sort the files based on their name
    image_files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

    # Create a list of NumPy arrays from the image files
    images = []
    for file in image_files:
        with Image.open(file) as img:
            # Add a text caption to the image indicating its file name
            draw = ImageDraw.Draw(img)
            draw.text((0, 0), os.path.basename(file))
            images.append(np.array(img))

    # Save the list of NumPy arrays as an animated GIF
    with imageio.get_writer(output_file, mode='I', fps=fps) as writer:
        for img in images:
            writer.append_data(img)

    print(f'Animated GIF created successfully: {output_file}')


In [13]:
create_gif('images', 'images.gif')

Animated GIF created successfully: images.gif


In [14]:
try:
    bucket.upload_file(Key=f"{DATASET_NAME}/images.gif", Filename='images.gif')
except:
    print("GIF don't exist.")

GIF don't exist.
